we build a classifier which can determine the numbers present in the pictures 

### Load tensorflow

In [0]:
import tensorflow as tf
tf.set_random_seed(42)

In [2]:
tf.__version__

'1.13.1'

### Collect Data

In [3]:
import keras

Using TensorFlow backend.


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


**Reading Data set**

In [0]:
import h5py
file='/content/drive/My Drive/AI_ML/Residency7 lab/SVHN_single_grey1.h5'
f= h5py.File(file,'r')
##data = pd.read_csv('/content/drive/My Drive/AI_ML/Residency7 lab/SVHN_single_grey1.h5')

**train, test and validation data split**

In [6]:
list(f.keys())

['X_test', 'X_train', 'X_val', 'y_test', 'y_train', 'y_val']

In [0]:
a_group_key = list(f.keys())

# Get the data
X_test = list(f[a_group_key[0]])
X_train= list(f[a_group_key[1]])
X_val=list(f[a_group_key[2]])
y_test=list(f[a_group_key[3]])
y_train=list(f[a_group_key[4]])
y_val=list(f[a_group_key[5]])

In [8]:
len(X_train)

42000

In [9]:
len(X_test)

18000

In [10]:
len(X_val)

60000

In [11]:
len(y_test)

18000

In [12]:
len(y_train)

42000

In [13]:
len(y_val)

60000

As shown above, 42000 is the train data and 18000 is the test data. There is a 70-30 train test split if the data size is considered to be 60000 excluding the validation data set

In [0]:
from numpy import array
x_train= array(X_train)
x_test=array(X_test)
y_train=array(y_train)
y_test=array(y_test)

### Convert train and test labels to one hot vectors

In [0]:
trainY = tf.keras.utils.to_categorical(y_train, num_classes=10)
testY = tf.keras.utils.to_categorical(y_test, num_classes=10)

**Normalize both the train and test image data from 0-255 to 0-1**

In [0]:
x_train/= 255
x_test/= 255

Reshape the train and test data

In [0]:
x_train = x_train.reshape(x_train.shape[0], 1024)
x_test = x_test.reshape(x_test.shape[0], 1024)

**KNN Algorithm**

In [30]:
## Import the Classifier.
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
for i in range(20,30):
## Instantiate the model with 5 neighbors. 
  knn = KNeighborsClassifier(n_neighbors=i,algorithm='brute', metric='euclidean')
## Fit the model on the training data.
  knn.fit(x_train, y_train)
#Predict the response for test dataset 
  y_pred = knn.predict(x_test)
#Import scikit-learn metrics module for accuracy calculation
  
# Model Accuracy, how often is the classifier correct?
  print("Accuracy for k ",i," is ",metrics.accuracy_score(y_test, y_pred))

Accuracy for k  20  is  0.5277777777777778
Accuracy for k  21  is  0.5290555555555555
Accuracy for k  22  is  0.5293333333333333
Accuracy for k  23  is  0.5298333333333334
Accuracy for k  24  is  0.5296111111111111
Accuracy for k  25  is  0.5297222222222222
Accuracy for k  26  is  0.5323888888888889
Accuracy for k  27  is  0.5322777777777777
Accuracy for k  28  is  0.532
Accuracy for k  29  is  0.5316666666666666


From the above it can be seen that the accuracy is more for k = 29

In [33]:
knn = KNeighborsClassifier(n_neighbors=29,algorithm='brute', metric='euclidean')
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.45      0.70      0.55      1814
           1       0.46      0.73      0.56      1828
           2       0.64      0.53      0.58      1803
           3       0.47      0.43      0.45      1719
           4       0.64      0.65      0.64      1812
           5       0.53      0.40      0.46      1768
           6       0.51      0.42      0.46      1832
           7       0.70      0.63      0.66      1808
           8       0.47      0.36      0.41      1812
           9       0.55      0.44      0.49      1804

   micro avg       0.53      0.53      0.53     18000
   macro avg       0.54      0.53      0.53     18000
weighted avg       0.54      0.53      0.53     18000



**Divide the data set into Train and test sets and Distinguish the feature and target set**

In [0]:
x_train= array(X_train)
x_test=array(X_test)

**normalse data, Initialize and build the model**

In [0]:
 #Initialize Sequential model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Reshape((1024,),input_shape=(32,32,)))
#Normalize the data
model.add(tf.keras.layers.BatchNormalization())
#model.add(tf.keras.layers.Dropout(0.3))
#Add First layer

#Add Dense Layer which provides 200 Outputs after applying relu
model.add(tf.keras.layers.Dense(200, activation='relu'))
model.add(tf.keras.layers.Dense(100, activation='relu'))
#Add Dense Layer which provides 100 Outputs after applying relu
model.add(tf.keras.layers.Dense(50, activation='relu'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

#Defining the optimizer

sgd_optimizer = tf.keras.optimizers.SGD(lr=0.01,clipnorm=1.)

#Compile the model - add Loss and Gradient Descent optimizer
model.compile(optimizer=sgd_optimizer, loss='categorical_crossentropy',metrics=['accuracy'])

### Execute the model using model.fit()

In [56]:
model.fit(x_train, trainY, 
          validation_data=(x_test, testY), 
          epochs=30,          batch_size=32)

Train on 42000 samples, validate on 18000 samples
Epoch 1/30
42000/42000 [==============================] - 8s 188us/sample - loss: 0.3979 - acc: 0.8741 - val_loss: 0.5653 - val_acc: 0.8457
Epoch 2/30
42000/42000 [==============================] - 8s 187us/sample - loss: 0.3951 - acc: 0.8746 - val_loss: 0.5612 - val_acc: 0.8469
Epoch 3/30
42000/42000 [==============================] - 8s 189us/sample - loss: 0.3909 - acc: 0.8739 - val_loss: 0.5748 - val_acc: 0.8461
Epoch 4/30
42000/42000 [==============================] - 8s 189us/sample - loss: 0.3884 - acc: 0.8768 - val_loss: 0.5589 - val_acc: 0.8488
Epoch 5/30
42000/42000 [==============================] - 8s 188us/sample - loss: 0.3838 - acc: 0.8778 - val_loss: 0.5719 - val_acc: 0.8468
Epoch 6/30
42000/42000 [==============================] - 8s 187us/sample - loss: 0.3848 - acc: 0.8772 - val_loss: 0.5739 - val_acc: 0.8482
Epoch 7/30
42000/42000 [==============================] - 8s 188us/sample - loss: 0.3846 - acc: 0.8776 - val_l

In [0]:
predictions=model.predict(x_test)

In [76]:
predictions[1]

array([6.2935054e-05, 4.4454480e-04, 8.4282631e-01, 2.3909115e-06,
       3.6010256e-08, 1.7101085e-05, 9.8714889e-05, 1.5634777e-01,
       6.0376378e-06, 1.9411245e-04], dtype=float32)

In [0]:
import numpy as np
final_predictions=[]
for x in range(len(predictions)):
#Get predicted number with highest probability
  predicted_num = np.argmax(predictions[x])
#Print the number
  final_predictions.append(predicted_num)
    

In [78]:
print("Firts 5 predictions : ")
for i in range(6):
  print("predicted value: ",final_predictions[i], " actual value: ",y_test[i])

Firts 5 predictions : 
predicted value:  1  actual value:  1
predicted value:  2  actual value:  7
predicted value:  2  actual value:  2
predicted value:  9  actual value:  9
predicted value:  0  actual value:  0
predicted value:  9  actual value:  9


## Review model

In [65]:
 model.evaluate(x_test,testY)

18000/18000 [==============================] - 1s 70us/sample - loss: 0.5810 - acc: 0.8503


[0.5810079094171524, 0.8502778]

In [79]:
print(metrics.classification_report(y_test, final_predictions))

              precision    recall  f1-score   support

           0       0.86      0.89      0.87      1814
           1       0.82      0.88      0.85      1828
           2       0.86      0.88      0.87      1803
           3       0.80      0.81      0.81      1719
           4       0.88      0.89      0.88      1812
           5       0.85      0.84      0.84      1768
           6       0.90      0.81      0.85      1832
           7       0.90      0.88      0.89      1808
           8       0.82      0.81      0.82      1812
           9       0.81      0.83      0.82      1804

   micro avg       0.85      0.85      0.85     18000
   macro avg       0.85      0.85      0.85     18000
weighted avg       0.85      0.85      0.85     18000



In [80]:
from sklearn.metrics import confusion_matrix 
confusion_matrix(y_test, final_predictions)

array([[1606,   32,   12,   23,   23,    5,   16,   13,   33,   51],
       [  33, 1604,   21,   30,   51,    5,   11,   34,   17,   22],
       [  18,   28, 1583,   26,   25,    5,    3,   46,   24,   45],
       [  17,   40,   41, 1389,   12,  102,   13,   29,   44,   32],
       [  23,   57,   27,   19, 1609,    7,   11,   10,   16,   33],
       [  15,   21,   18,   90,   10, 1479,   41,   10,   34,   50],
       [  57,   23,   14,   23,   39,   65, 1475,   11,   94,   31],
       [  15,   74,   60,   22,   15,    9,    5, 1582,    8,   18],
       [  22,   42,   29,   58,   22,   29,   55,    9, 1476,   70],
       [  59,   25,   34,   49,   22,   39,   12,   18,   44, 1502]])

**Observations**

From the above values of knn and neural network, we can clearly see there is a very bigchange in accuracy and classification metrics.
KNN  : 53% accurate
DNN: 85% accurate



The classification metrics :
for KNN at an average :
precision : 54, Recall : 53
For DNN atan average:
precision : 85, recall : 85

Hence we can conclude, the DNN works much better than traditional knn in the real time data like this